In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
""" # URL de la página que deseas hacer scraping (reemplaza con la URL correcta)
url = 'https://www.kemik.gt/tienda-en-linea/electronicos?page=2'

# Realizar la solicitud GET a la página web
response = requests.get(url)

# Comprobar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar productos y precios con clases múltiples
    products = soup.find_all('div', class_='product-summary_title__RSvpT')
    prices = soup.find_all('div', class_=['price_price__xQt90', 'product-summary_price__wQCFs product-summary_sale___6b6c'])

    # Imprimir los productos y precios
    for product, price in zip(products, prices):
        print(f"Producto: {product.get_text(strip=True)} - Precio: {price.get_text(strip=True)}")
else:
    print(f"Error al acceder a la página: {response.status_code}") """

' # URL de la página que deseas hacer scraping (reemplaza con la URL correcta)\nurl = \'https://www.kemik.gt/tienda-en-linea/electronicos?page=2\'\n\n# Realizar la solicitud GET a la página web\nresponse = requests.get(url)\n\n# Comprobar si la solicitud fue exitosa\nif response.status_code == 200:\n    # Parsear el contenido HTML\n    soup = BeautifulSoup(response.content, \'html.parser\')\n\n    # Encontrar productos y precios con clases múltiples\n    products = soup.find_all(\'div\', class_=\'product-summary_title__RSvpT\')\n    prices = soup.find_all(\'div\', class_=[\'price_price__xQt90\', \'product-summary_price__wQCFs product-summary_sale___6b6c\'])\n\n    # Imprimir los productos y precios\n    for product, price in zip(products, prices):\n        print(f"Producto: {product.get_text(strip=True)} - Precio: {price.get_text(strip=True)}")\nelse:\n    print(f"Error al acceder a la página: {response.status_code}") '

In [3]:
nombre_producto = []
precio = []
link = []

for i in range(1,26):
    # Crear un header para la solicitud que simule un navegador
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'DNT': '1',  # Do Not Track request header
    }
    
    # URL de la página que deseas hacer scraping (reemplaza con la URL correcta)
    url = 'https://www.kemik.gt/tienda-en-linea/electronicos?page={i}'.format(i = i)
    # Realizar la solicitud GET a la página web
    response = requests.get(url, headers=headers)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Obteniendo el nombre del producto
    producto = soup.find_all('div', class_='product-summary_title__RSvpT')
    for products in producto:
        prod = products.get_text(strip=True)
        nombre_producto.append(prod)
        
    # Obteniendo los precios de oferta y normales
    prices = soup.find_all('div', class_=['product-summary_prices__qbBVd',])
    for price in prices:
        pre = price.get_text(strip=True)
        precio.append(pre)

    # Extraer y mostrar los href de cada enlace
    enlaces = soup.find_all('a', class_='product-summary_product-summary__dYYlE product-search-pages_product__YoKp1 product-summary_shown__6Iu_Q')
    for enlace in enlaces:
        href = enlace.get('href')
        link_pagina = 'https://www.kemik.gt'+ href
        link.append(link_pagina)
        

In [4]:
df_kemik = pd.DataFrame()
df_kemik["nombre_producto"] = pd.Series(nombre_producto)
df_kemik["precios"] = pd.Series(precio)
df_kemik["url_articulo"] = pd.Series(link)

In [5]:
# Función para dividir después de la segunda aparición de "Q"
def separar_precios(precio_str):
    # Separar las cadenas de precios por el espacio antes de cada "Q"
    partes = precio_str.split("Q")
    
    # Si hay más de una aparición de "Q", crear dos columnas
    if len(partes) > 2:
        columna1 = "Q " + partes[1].strip()
        columna2 = "Q " + partes[2].strip()
        return (columna1, columna2)
    else:
        return (precio_str.strip(), "")

In [6]:
# Aplicar la función a la columna 'precios' y expandir el resultado en dos nuevas columnas
df_kemik[['precio_oferta', 'precio_normal']] = df_kemik['precios'].apply(separar_precios).apply(pd.Series)

In [7]:
# Función para eliminar la letra "Q" y convertir a número
def convertir_a_numero(precio_str):
    # Usar expresiones regulares para eliminar la "Q" y las comas
    return float(re.sub(r'[Q,]', '', precio_str))

# Aplicar la función a las columnas precio_1 y precio_2, y convertir a números
df_kemik['precio_oferta'] = df_kemik['precio_oferta'].replace('', '0').apply(convertir_a_numero)
df_kemik['precio_normal'] = df_kemik['precio_normal'].replace('', '0').apply(convertir_a_numero)

df_kemik["porcentaje_oferta"] = df_kemik.apply(lambda x: 100 - ((x['precio_oferta'] * 100) / (x['precio_normal'])) if x['precio_normal'] > 0 
                                               else 0, axis = 1)

In [9]:
df_kemik.to_excel("area_tecnologia_kemik.xlsx", index = False)